In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from ta_lib.core.api import load_yml,create_context
from ta_lib.data_utils import offline_constants

In [3]:
config_path = '../conf/config.yml'
cfg=create_context(config_path)

In [4]:
date_ranges = pd.date_range('2018-08-01','2018-10-01',freq='MS')

In [5]:
import os.path as op

In [ ]:
# required_columns = ["pos_ord_nu","pos_itm_tot_qt","pos_itm_grss_unt_prc_am","mcd_gbal_lcat_id_nu","pos_busn_dt","terr_pos_trn_typ_cd","sld_menu_itm_id"]
import os.path as op
date_ranges = pd.date_range('2018-08-01','2018-10-01',freq='MS')
df_final = pd.DataFrame()
for date_ in date_ranges:
    path_ = op.join(cfg.offline['OFFLINE_RAW'].format(**cfg.offline),f"{date_.strftime('%Y%m%d')}")
    item_data=pd.read_parquet(path_,columns = required_columns)
    df_final = df_final.append(item_data)

In [25]:
dig_tran =pd.read_parquet(op.join(cfg.offline['DIGITAL_TRNS'].format(**cfg.offline),f"month_id={pd.to_datetime('2020-03-01').strftime('%Y%m%d')}"))

In [6]:
date_ranges

DatetimeIndex(['2020-03-01'], dtype='datetime64[ns]', freq='MS')

In [ ]:
def process_offline_item(grouping = True,filter ='digital'/'offline'/'both'):
    date_ranges = pd.date_range('2020-03-01','2020-05-01',freq='MS')
    df_item_master = pd.read_xlsx(cfg.offline["ITEM_MASTER"].format(**cfg.offline))[]
    for date_ in date_ranges:
        path_ = op.join(cfg.offline['OFFLINE_RAW'].format(**cfg.offline),f"{date_.strftime('%Y%m%d')}")
        item_data=pd.read_parquet(path_,columns = required_columns)
        path_dig_ = op.join(cfg.offline['DIGITAL_TRNS'].format(**cfg.offline),f"month_id={date_.strftime('%Y%m%d')}")
        dig_tran_data =pd.read_parquet(path_dig_)


In [ ]:
def add_gc_count(df): 
    
    refund_trns = ['014', '114']
    ref_trn_count = df.query('terr_pos_trn_typ_cd in @refund_trns')[['pos_busn_dt', 'mcd_gbal_lcat_id_nu', 'pos_ord_nu']].drop_duplicates().shape[0]
    tot_trn_count = df[['pos_busn_dt', 'mcd_gbal_lcat_id_nu', 'pos_ord_nu']].drop_duplicates().shape[0]
    
    return tot_trn_count-2*ref_trn_count

In [6]:
import numpy as np

In [7]:
item_df = pd.read_csv(cfg.offline["ITEM_MASTER"].format(**cfg.offline))[['sld_menu_itm_id', 'Group_id']]
item_df['Master_id'] = np.where(item_df['Group_id'].isna(), item_df['sld_menu_itm_id'], item_df['Group_id'])
dict_master = dict(zip(item_df["sld_menu_itm_id"],item_df['Master_id']))
del item_df['Group_id']

In [10]:
df_offer_day = pd.read_parquet(cfg.offline["PROMO_DAY"].format(**cfg.offline))

ArrowIOError: Invalid parquet file. Corrupt footer.

In [8]:
date_ = date_ranges[0]

In [10]:
import gc

In [9]:
grouper_cols = ['Master_id', 'pos_busn_dt']

# Loading dumped GDW data
path_ = op.join(cfg.offline['OFFLINE_RAW'].format(**cfg.offline),f"{date_.strftime('%Y%m%d')}")
df = pd.read_parquet(path_).iloc[:10000,]
df['Master_id'] = df["sld_menu_itm_id"].map(dict_master)
df['Master_id'] = np.where(df['Master_id'].isna(), df['sld_menu_itm_id'], df['Master_id'])
print('Done with loading data')
# Altering Cmobo price to get WAP if needed
alc_df = df[(df['cmbo_pren_sld_menu_itm_id'] == -1)].reset_index(drop=True)
cmbo_df = df[(df['cmbo_pren_sld_menu_itm_id'] != -1)].reset_index(drop=True)

del df
gc.collect()


Done with loading data


NameError: name 'gc' is not defined

In [11]:
cmbo_df["pos_itm_grss_unt_prc_am"] = cmbo_df.groupby(['pos_busn_dt', 
                              'mcd_gbal_lcat_id_nu',
                              'pos_ord_nu', 
                              'cmbo_pren_sld_menu_itm_id'])[['pos_itm_grss_unt_prc_am']].transform("sum")


In [12]:
cmbo_df.head()

,pos_ord_nu,pos_itm_tot_qt,pos_itm_grss_unt_prc_am,pos_tot_disc_am,mcd_gbal_lcat_id_nu,cmbo_pren_sld_menu_itm_id,pos_busn_dt,terr_pos_trn_typ_cd,sld_menu_itm_id,Master_id
0,304206479,1,200.0,0.0,195500585932,8784,2018-08-01,000,8767,8767.0
1,304206479,1,200.0,0.0,195500585932,8784,2018-08-01,000,8784,8784.0
2,304206479,1,50.0,0.0,195500585932,3045,2018-08-01,000,3045,3010.0
3,454813201,1,70.0,0.0,195500885643,8784,2018-08-01,000,8813,8813.0
4,341748636,1,105.0,0.0,195500188942,8784,2018-08-01,100,8821,8821.0


In [13]:
cmbo_df = cmbo_df[cmbo_df['cmbo_pren_sld_menu_itm_id'] == cmbo_df['sld_menu_itm_id']]
cmbo_df["pos_itm_tot_qt"] = cmbo_df.groupby(['pos_busn_dt', 
                              'mcd_gbal_lcat_id_nu',
                              'pos_ord_nu', 
                              'cmbo_pren_sld_menu_itm_id'])[['pos_itm_tot_qt']].transform("sum")
cmbo_df['pos_itm_tot_qt'].fillna(1, inplace=True)
cmbo_df['pos_itm_grss_unt_prc_am'] = cmbo_df['pos_itm_grss_unt_prc_am']/cmbo_df['pos_itm_tot_qt']


In [ ]:
# Merging Store Master file
item_df = pd.read_csv(cfg.offline["ITEM_MASTER"].format(**cfg.offline))[['sld_menu_itm_id', 'Group_id']]
item_df['Master_id'] = np.where(item_df['Group_id'].isna(), item_df['sld_menu_itm_id'], item_df['Group_id'])
dict_master = dict(zip(item_df["sld_menu_itm_id"],item_df['Master_id']))
del item_df['Group_id']
    
for date_ in date_ranges:
    
    grouper_cols = ['Master_id', 'pos_busn_dt']
    
    # Loading dumped GDW data
    path_ = op.join(cfg.offline['OFFLINE_RAW'].format(**cfg.offline),f"{date_.strftime('%Y%m%d')}")
    df = pd.read_parquet(path_)
    df['Master_id'] = df["sld_menu_itm_id"].map(dict_master)
    df['Master_id'] = np.where(df['Master_id'].isna(), df['sld_menu_itm_id'], df['Master_id'])
    
    # Getting Trn key
#     df['trn_key'] = df['pos_busn_dt'].astype('str') + '-' + df['mcd_gbal_lcat_id_nu'].astype('stre') + '-' + \
#                     df['pos_ord_nu'].astype('str')
        
    print('Donw ith loading data')
    # Altering Cmobo price to get WAP if needed
    alc_df = df[(df['cmbo_pren_sld_menu_itm_id'] == -1)].reset_index(drop=True)
    cmbo_df = df[(df['cmbo_pren_sld_menu_itm_id'] != -1)].reset_index(drop=True)

    del df
    gc.collect()

    cmbo_df["pos_itm_grss_unt_prc_am"] = cmbo_df.groupby(['pos_busn_dt', 
                                  'mcd_gbal_lcat_id_nu',
                                  'pos_ord_nu', 
                                  'cmbo_pren_sld_menu_itm_id'])[['pos_itm_grss_unt_prc_am']].transform("sum")
    cmbo_df = cmbo_df[cmbo_df['cmbo_pren_sld_menu_itm_id'] == cmbo_df['sld_menu_itm_id']]
    cmbo_df["pos_itm_tot_qt"] = cmbo_df.groupby(['pos_busn_dt', 
                                  'mcd_gbal_lcat_id_nu',
                                  'pos_ord_nu', 
                                  'cmbo_pren_sld_menu_itm_id'])[['pos_itm_tot_qt']].transform("sum")
    cmbo_df['pos_itm_tot_qt'].fillna(1, inplace=True)
    cmbo_df['pos_itm_grss_unt_prc_am'] = cmbo_df['pos_itm_grss_unt_prc_am']/cmbo_df['pos_itm_tot_qt']
    

    df = pd.concat([alc_df, cmbo_df], axis=0).reset_index(drop=True)

    # Removing Child Items
    df = df[(df['cmbo_pren_sld_menu_itm_id'] == -1)|(df['cmbo_pren_sld_menu_itm_id'] == df['sld_menu_itm_id'])].reset_index(drop=True)
        
    # Loading digital trn data to remove those trns
    path_dig_ = op.join(cfg.offline['DIGITAL_TRNS'].format(**cfg.offline),f"month_id={date_.strftime('%Y%m%d')}")
    if path_dig_:
        dig_df = pd.read_parquet(path_dig_,columns=['order_key', 'mcd_gbal_lcat_id_nu'])
        dig_df['pos_ord_nu'] = dig_df['order_key'].apply(lambda x : x.split(':')[1]).astype('int64')
        
        dig_df['mcd_gbal_lcat_id_nu'] = dig_df['mcd_gbal_lcat_id_nu'].astype('int64')
        
        dig_df['digital_flag'] = 1
        del dig_df['order_key']
        del dig_df['store_number']
        del dig_df['busn_lcat_id_nu']
        
        df = df.merge(dig_df, on=['mcd_gbal_lcat_id_nu', 'pos_ord_nu'], how='left')
        print('Donw ith digital merge')
        
        # Clearing space
        del dig_df
        del store_df
        del item_df
        gc.collect()
        
        if 'Digital' in 'Digital' :
            dig_temp_df = df[(~df['digital_flag'].isna())].reset_index(drop=True)
            
            dig_res_df = dig_temp_df.groupby(grouper_cols)[['pos_itm_tot_qt']].sum().reset_index()
            dig_res_df = dig_res_df.merge(dig_temp_df.groupby(grouper_cols)[['trn_key']].nunique().reset_index(), on=grouper_cols)
            dig_res_df = dig_res_df.merge(dig_temp_df[(dig_temp_df['terr_pos_trn_typ_cd'].isin(['014', '114']))].groupby(grouper_cols)[['trn_key']].nunique().reset_index(),
                                          on=grouper_cols)
            
            # Getting WAP
            dig_temp_df['mult'] = dig_temp_df['pos_itm_grss_unt_prc_am']*dig_temp_df['pos_itm_tot_qt']
            dig_wap_df = dig_temp_df.groupby(grouper_cols)[['mult', 'pos_itm_tot_qt']].sum().reset_index()
            dig_wap_df['WAP'] = (dig_wap_df['mult']/dig_wap_df['pos_itm_tot_qt']).round(2)
            del dig_wap_df['mult']
            del dig_wap_df['pos_itm_tot_qt']
            del dig_wap_df['pos_itm_grss_unt_prc_am']
            
            dig_res_df = dig_res_df.merge(dig_wap_df, on=grouper_cols)
            print('Done ith digital result')
            
        if 'Offline' in 'Offline' :
            off_temp_df = df[(df['digital_flag'].isna())].reset_index(drop=True)
            off_res_df = off_temp_df.groupby(grouper_cols)[['pos_itm_tot_qt']].sum().reset_index()
            off_res_df = off_res_df.merge(off_temp_df.groupby(grouper_cols)[['trn_key']].nunique().reset_index(), on=grouper_cols)
            off_res_df = off_res_df.merge(off_temp_df[(off_temp_df['terr_pos_trn_typ_cd'].isin(['014', '114']))].groupby(grouper_cols)[['trn_key']].nunique().reset_index(),
                                          on=grouper_cols)
            
            # Getting WAP
            off_temp_df['mult'] = off_temp_df['pos_itm_grss_unt_prc_am']*off_temp_df['pos_itm_tot_qt']
            off_wap_df = off_temp_df.groupby(grouper_cols)[['mult', 'pos_itm_tot_qt']].sum().reset_index()
            off_wap_df['WAP'] = (off_wap_df['mult']/off_wap_df['pos_itm_tot_qt']).round(2)
            del off_wap_df['mult']
            del off_wap_df['pos_itm_tot_qt']
            del off_wap_df['pos_itm_grss_unt_prc_am']
            
            
            off_res_df = off_res_df.merge(off_wap_df, on=grouper_cols)
        
        if 'Overall' in 'Overall' :
            overall_temp_df = df.copy()
            overall_temp_df = overall_temp_df.groupby(grouper_cols)[['pos_itm_tot_qt']].sum().reset_index()
            overall_temp_df = overall_temp_df.merge(overall_temp_df.groupby(grouper_cols)[['trn_key']].nunique().reset_index(), on=grouper_cols)
            overall_temp_df = overall_temp_df.merge(overall_temp_df[(overall_temp_df['terr_pos_trn_typ_cd'].isin(['014', '114']))].groupby(grouper_cols)[['trn_key']].nunique().reset_index(),
                                          on=grouper_cols)
            
            # Getting WAP
            overall_temp_df['mult'] = overall_temp_df['pos_itm_grss_unt_prc_am']*overall_temp_df['pos_itm_tot_qt']
            overall_temp_df = overall_temp_df.groupby(grouper_cols)[['mult', 'pos_itm_tot_qt']].sum().reset_index()
            overall_temp_df['WAP'] = (overall_temp_df['mult']/overall_temp_df['pos_itm_tot_qt']).round(2)
            del overall_temp_df['mult']
            del overall_temp_df['pos_itm_tot_qt']
            del overall_temp_df['pos_itm_grss_unt_prc_am']
            
            
            overall_temp_df = overall_temp_df.merge(overall_temp_df, on=grouper_cols)
            print('Done ith overall result')


In [12]:
import numpy as np

In [ ]:
def process_offline_item(grouping = True,filter ='digital'/'offline'/'both') - Akash

	read parquet partition from EFS - sld_menu_itm_id, pos_ord_nu, date , store, quantity, grossprice
	if grouping
		apply grouping
	elif:
		no grouping
		
	1. over items
	2. offline
		ignore digital_trans -> order_key and store_number
	3. digitl
		
	else: 
		digital records from here https://share.mcd.com/personal/unique_agiwal_us_mcd_com/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Funique%5Fagiwal%5Fus%5Fmcd%5Fcom%2FDocuments%2Ftrans%5Fids&FolderCTID=0x012000F68E2DC1F54760438C553D1D374488E9
	
	df = 
	df2 = df
	calculate wap on df2
	calculate
		item_gc
		qty
		wap - calculate wap_offline()
		offer
		processed/overall/item_level/sld_menu_item_id={}/month_id={}
		processed/overall/item_group_level/
		processed/offline/item_level/
		processed/offline/item_group_level/
		processed/digital/item_level/grouped
		processed/digital/item_group_level/

In [9]:
pd.read_parquet(r"M:\dev\PricingEngines\DiscountEngines\Russia\data\raw\offline\20180801\data.parquet", columns=["pos_ord_nu","pos_itm_tot_qt","pos_itm_grss_unt_prc_am","mcd_gbal_lcat_id_nu","pos_busn_dt","terr_pos_trn_typ_cd","sld_menu_itm_id"])

,pos_ord_nu,pos_itm_tot_qt,pos_itm_grss_unt_prc_am,pos_tot_disc_am,mcd_gbal_lcat_id_nu,cmbo_pren_sld_menu_itm_id,pos_busn_dt,terr_pos_trn_typ_cd,sld_menu_itm_id
0,189177474,1,50.0,0.0,195500777559,-1,2018-08-01,000,2302
1,304206479,1,20.0,0.0,195500585932,-1,2018-08-01,000,2372
2,189177474,1,50.0,0.0,195500777559,-1,2018-08-01,000,5005
3,14388657,1,145.0,0.0,195500194714,-1,2018-08-01,000,7389
4,31220586,1,95.0,0.0,195500189062,-1,2018-08-01,000,4118
...,...,...,...,...,...,...,...,...,...
119068137,166976956,1,50.0,0.0,195500388229,-1,2018-08-22,000,2010
119068138,281023449,1,20.0,0.0,195500769806,-1,2018-08-22,000,2364
119068139,1054233429,2,50.0,0.0,195500388229,-1,2018-08-22,000,2010
119068140,1054233315,1,50.0,0.0,195500388229,-1,2018-08-22,000,2010
